# Домашнее задание № 2. Мешок слов

## Задание 1 (3 балла)

У векторайзеров в sklearn есть встроенная токенизация на регулярных выражениях. Найдите способо заменить её на кастомную токенизацию

Обучите векторайзер с дефолтной токенизацией и с токенизацией razdel.tokenize. Обучите классификатор (любой) с каждым из векторизаторов. Сравните метрики и выберете победителя. 

(в вашей тетрадке должен быть код обучения и все метрики; если вы сдаете в .py файлах то сохраните полученные метрики в отдельном файле или в комментариях)

In [1]:
import pandas as pd
import math
from razdel import tokenize
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, accuracy_score
pd.options.mode.chained_assignment = None 

In [2]:
data = pd.read_csv('labeled.csv')

In [3]:
data.head()

,comment,toxic
0,"Верблюдов-то за что? Дебилы, бл...\n",1.0
1,"Хохлы, это отдушина затюканого россиянина, мол...",1.0
2,Собаке - собачья смерть\n,1.0
3,"Страницу обнови, дебил. Это тоже не оскорблени...",1.0
4,"тебя не убедил 6-страничный пдф в том, что Скр...",1.0


Модель с токенизацией sklearn:

In [4]:
train, test = train_test_split(data, test_size=0.1, shuffle=True)

In [5]:
train.reset_index(inplace=True)
test.reset_index(inplace=True)

In [6]:
# векторизация + токенизация sklearn
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(train.comment)
X_test = vectorizer.transform(test.comment) 

In [7]:
y = train.toxic.values
y_test = test.toxic.values

In [8]:
log_reg = LogisticRegression(C=0.1, class_weight='balanced')

In [9]:
log_reg.fit(X, y)

LogisticRegression(C=0.1, class_weight='balanced')

In [10]:
preds = log_reg.predict(X_test)

In [11]:
print(classification_report(y_test, preds, zero_division=0))

              precision    recall  f1-score   support

         0.0       0.90      0.84      0.87       927
         1.0       0.74      0.84      0.79       515

    accuracy                           0.84      1442
   macro avg       0.82      0.84      0.83      1442
weighted avg       0.84      0.84      0.84      1442



Модель с токенизацией razdel:

In [12]:
def razdel_tokenize(data):
    tokens = list(tokenize(str(data)))
    return [_.text for _ in tokens]

In [13]:
vectorizer = CountVectorizer(tokenizer = razdel_tokenize)
X_2 = vectorizer.fit_transform(train.comment)

In [14]:
X_test_2 = vectorizer.transform(test.comment) 

In [15]:
log_reg.fit(X_2, y)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(C=0.1, class_weight='balanced')

In [16]:
preds = log_reg.predict(X_test_2)

In [17]:
print(classification_report(y_test, preds, zero_division=0))

              precision    recall  f1-score   support

         0.0       0.90      0.83      0.86       927
         1.0       0.73      0.83      0.78       515

    accuracy                           0.83      1442
   macro avg       0.82      0.83      0.82      1442
weighted avg       0.84      0.83      0.83      1442



Результат: кажется, что модель с CountVectorizer работает чуть лучше 

## Задание 2 (3 балла)

Преобразуйте таблицу с абсолютными частотностями в семинарской тетрадке в таблицу с tfidf значениями. (Таблица - https://i.ibb.co/r5Nc2HC/abs-bow.jpg) Формула tfidf есть в семинаре на картнике с пояснениями на английском. 
Считать нужно в питоне. Формат итоговой таблицы может быть любым, главное, чтобы был код и можно было воспроизвести вычисления. 

In [18]:
data = {'sent': ['я и ты', 'ты и я', 'я, я и только я', 'только не я', 'он'],
        'я': [0.33, 0.33, 0.60, 0.33, 0],
       'ты': [0.33,0.33,0,0,0],
       'и': [0.33, 0.33, 0.20, 0, 0],
       'только': [0,0, 0.20, 0.33, 0],
       'не': [0,0,0, 0.33, 0],
       'он': [0,0,0,0,1]}

In [19]:
df = pd.DataFrame(data)
df

,sent,я,ты,и,только,не,он
0,я и ты,0.33,0.33,0.33,0.00,0.00,0
1,ты и я,0.33,0.33,0.33,0.00,0.00,0
2,"я, я и только я",0.60,0.00,0.20,0.20,0.00,0
3,только не я,0.33,0.00,0.00,0.33,0.33,0
4,он,0.00,0.00,0.00,0.00,0.00,1


In [20]:
df_2 = pd.DataFrame(data)

In [21]:
def idf(val, sent):
    tf = float(df[val][sent])
    n = float(len(df))
    df_x = float(len(df[df[val] != 0]))
    for_log = n/df_x
    log = math.log(for_log)
    final = tf * log
    return final

In [22]:
df_2['я'][0] = idf('я', 0)
df_2['я'][1] = idf('я', 1)
df_2['я'][2] = idf('я', 2)
df_2['я'][3] = idf('я', 3)
df_2['я'][4] = idf('я', 4)

In [23]:
df_2['ты'][0] = idf('ты', 0)
df_2['ты'][1] = idf('ты', 1)
df_2['ты'][2] = idf('ты', 2)
df_2['ты'][3] = idf('ты', 3)
df_2['ты'][4] = idf('ты', 4)

In [24]:
df_2['и'][0] = idf('и', 0)
df_2['и'][1] = idf('и', 1)
df_2['и'][2] = idf('и', 2)
df_2['и'][3] = idf('и', 3)
df_2['и'][4] = idf('и', 4)

In [25]:
df_2['только'][0] = idf('только', 0)
df_2['только'][1] = idf('только', 1)
df_2['только'][2] = idf('только', 2)
df_2['только'][3] = idf('только', 3)
df_2['только'][4] = idf('только', 4)

In [26]:
df_2['не'][0] = idf('не', 0)
df_2['не'][1] = idf('не', 1)
df_2['не'][2] = idf('не', 2)
df_2['не'][3] = idf('не', 3)
df_2['не'][4] = idf('не', 4)

In [27]:
df_2['он'][0] = idf('он', 0)
df_2['он'][1] = idf('он', 1)
df_2['он'][2] = idf('он', 2)
df_2['он'][3] = idf('он', 3)
df_2['он'][4] = idf('он', 4)

In [28]:
df_2

,sent,я,ты,и,только,не,он
0,я и ты,0.073637,0.302376,0.168572,0.000000,0.000000,0
1,ты и я,0.073637,0.302376,0.168572,0.000000,0.000000,0
2,"я, я и только я",0.133886,0.000000,0.102165,0.183258,0.000000,0
3,только не я,0.073637,0.000000,0.000000,0.302376,0.531115,0
4,он,0.000000,0.000000,0.000000,0.000000,0.000000,1


## Задание 3 (2 балла)

Обучите 2 любых разных классификатора из семинара. Предскажите токсичность для текстов из тестовой выборки (используйте одну и ту же выборку для обоих классификаторов) и найдите 10 самых токсичных для каждого из классификаторов. Сравните получаемые тексты - какие тексты совпадают, какие отличаются, правда ли тексты токсичные?

Требования к моделям:   
а) один классификатор должен использовать CountVectorizer, другой TfidfVectorizer  
б) у векторазера должны быть вручную заданы как минимум 5 параметров (можно ставить разные параметры tfidfvectorizer и countvectorizer)  
в) у классификатора должно быть задано вручную как минимум 2 параметра (по возможности)  
г)  f1 мера каждого из классификаторов должна быть минимум 0.75  

*random_seed не считается за параметр

In [29]:
y = train.toxic.values
y_test = test.toxic.values

In [30]:
vectorizer_cv = CountVectorizer(tokenizer = razdel_tokenize, ngram_range = (1,2), max_df = 0.5, min_df = 0.0, binary = True)
X_cv = vectorizer_cv.fit_transform(train.comment)
X_cv_test = vectorizer_cv.transform(test.comment)

In [31]:
log_reg = LogisticRegression(C=0.1, class_weight='balanced')
log_reg.fit(X_cv, y)
preds = log_reg.predict(X_cv_test)

print(classification_report(y_test, preds))

              precision    recall  f1-score   support

         0.0       0.88      0.85      0.87       927
         1.0       0.75      0.79      0.77       515

    accuracy                           0.83      1442
   macro avg       0.82      0.82      0.82      1442
weighted avg       0.83      0.83      0.83      1442



Классификатор с Tfidf: 

In [32]:
vectorizer_tv = TfidfVectorizer(tokenizer = razdel_tokenize, min_df=1, max_df=1.0, analyzer = 'word', ngram_range = (0,1))
X_tv = vectorizer_tv.fit_transform(train.comment)
X_tv_test = vectorizer_tv.transform(test.comment) 

In [33]:
clf = KNeighborsClassifier(n_neighbors=15, weights='distance', metric='minkowski', algorithm = 'auto')
clf.fit(X_tv, y)
preds = clf.predict(X_tv_test)

print(classification_report(y_test, preds))

              precision    recall  f1-score   support

         0.0       0.79      0.88      0.84       927
         1.0       0.74      0.59      0.65       515

    accuracy                           0.78      1442
   macro avg       0.77      0.74      0.75      1442
weighted avg       0.77      0.78      0.77      1442



## *Задание 4 (2 балла)

Для классификаторов LogisticRegression и Random Forest найдите способ извлечь важность признаков для предсказания токсичного класса. Сопоставьте полученные числа со словами (или нграммами) в словаре и найдите топ - 5 "токсичных" слов для каждого из классификаторов. 

Важное требование: в топе не должно быть стоп-слов. Для этого вам нужно будет правильным образом настроить векторизацию.